En el siguiente notebook vamos a trabajar con PySpark. En él, vamos a usar las transformaciones y las acciones para analizar un pequeño conjunto de datos. 

El primer paso que debemos dar es conectar Google Colab con nuestro Google Drive. Para ello, lanzaremos el siguiente trozo de código.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!ls

drive  sample_data


Una vez hemos conectado Google Drive, pasamos a instalar Apache Spark en nuestro notebook. En concreto, vamos a trabajar con la versión 3.0.3.

In [3]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [4]:
!wget -q http://apache.mirrors.pair.com/spark/spark-3.0.3/spark-3.0.3-bin-hadoop2.7.tgz

In [5]:
!ls


drive  sample_data  spark-3.0.3-bin-hadoop2.7.tgz


In [6]:
!tar xf spark-3.0.3-bin-hadoop2.7.tgz

Además de la instalación habitual, hay un paso más que debemos dar, y es la instalación de la librería *findspark*. Esta librería nos permitirá encontrar la instalación de Apache Spark en nuestro sistema.

Establecemos también un par de variables de entorno.

In [7]:
!pip install -q findspark

In [8]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.3-bin-hadoop2.7"

In [9]:
import findspark
findspark.init()

In [10]:
findspark.find()

'/content/spark-3.0.3-bin-hadoop2.7'

Una vez tenemos instalado nuestro framework, es momento de crear nuestra variable SparkSession, de la cual parten la mayoría de funcionalidades de SparkSQL

In [11]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Pyspark_SQL")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [12]:
spark

### Lectura de datos

Ahora sí, podemos comenzar a trabajar con SparkSQL. Comenzaremos leyendo nuestro conjunto de datos, el cual contiene información de valoraciones de restaurantes.

In [13]:
!pwd

/content


In [14]:
import pandas as pd

data_path = '/content/drive/MyDrive/TokioSchool/data/'

data = spark.read.options(inferSchema='True',delimiter=';', header=True).csv(data_path + 'restaurantes.csv')
data.take(2)

[Row(Index=0, Name="Martine of Martine's Table", City='Amsterdam', CuisineStyle="['French', 'Dutch', 'European']", Ranking=1.0, Rating=5.0, PriceRange='$$ - $$$', NumberOfReviews=136.0),
 Row(Index=1, Name='De Silveren Spiegel', City='Amsterdam', CuisineStyle="['Dutch', 'European', 'Vegetarian Friendly', 'Gluten Free Options']", Ranking=2.0, Rating=4.5, PriceRange='$$$$', NumberOfReviews=812.0)]

In [15]:
data

DataFrame[Index: int, Name: string, City: string, CuisineStyle: string, Ranking: double, Rating: double, PriceRange: string, NumberOfReviews: double]

### Análisis inicial

Podemos ver la estructura de nuestros datos usando la función *printSchema*. Luego, mostramos el contenido del fichero en formato *dataframe* usando el método *show*

In [16]:
data.printSchema()

root
 |-- Index: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- City: string (nullable = true)
 |-- CuisineStyle: string (nullable = true)
 |-- Ranking: double (nullable = true)
 |-- Rating: double (nullable = true)
 |-- PriceRange: string (nullable = true)
 |-- NumberOfReviews: double (nullable = true)



In [17]:
data.show(3)

+-----+--------------------+---------+--------------------+-------+------+----------+---------------+
|Index|                Name|     City|        CuisineStyle|Ranking|Rating|PriceRange|NumberOfReviews|
+-----+--------------------+---------+--------------------+-------+------+----------+---------------+
|    0|Martine of Martin...|Amsterdam|['French', 'Dutch...|    1.0|   5.0|  $$ - $$$|          136.0|
|    1| De Silveren Spiegel|Amsterdam|['Dutch', 'Europe...|    2.0|   4.5|      $$$$|          812.0|
|    2|             La Rive|Amsterdam|['Mediterranean',...|    3.0|   4.5|      $$$$|          567.0|
+-----+--------------------+---------+--------------------+-------+------+----------+---------------+
only showing top 3 rows



In [18]:
data.first()

Row(Index=0, Name="Martine of Martine's Table", City='Amsterdam', CuisineStyle="['French', 'Dutch', 'European']", Ranking=1.0, Rating=5.0, PriceRange='$$ - $$$', NumberOfReviews=136.0)

Para usar solo algunas columnas podemos usar el método *select*

In [19]:
temp = data.select('City', 'Rating', 'NumberOfReviews')
temp.show(5)

+---------+------+---------------+
|     City|Rating|NumberOfReviews|
+---------+------+---------------+
|Amsterdam|   5.0|          136.0|
|Amsterdam|   4.5|          812.0|
|Amsterdam|   4.5|          567.0|
|Amsterdam|   5.0|          564.0|
|Amsterdam|   4.5|          316.0|
+---------+------+---------------+
only showing top 5 rows



In [20]:
temp

DataFrame[City: string, Rating: double, NumberOfReviews: double]

### Estadísticas

In [21]:
data.count()

125527

In [ ]:
data.describe().show()

+-------+------------------+-------------------------+---------+--------------------+-----------------+------------------+----------+------------------+
|summary|             Index|                     Name|     City|        CuisineStyle|          Ranking|            Rating|PriceRange|   NumberOfReviews|
+-------+------------------+-------------------------+---------+--------------------+-----------------+------------------+----------+------------------+
|  count|            125527|                   125527|   125527|               94176|           115876|            115897|     77672|            108183|
|   mean|3974.6861312705632|                 Infinity|     null|                null| 3657.46397873589|3.9874414350673444|      null|125.18498285312849|
| stddev|  4057.68769819953|                      NaN|     null|                null|3706.255301231757|0.6788135754644803|      null|310.83331055968836|
|    min|                 0|          !Bebop daslokal|Amsterdam|['Afghani', 'Midd.

También podemos sacar algunas estadísticas concretas como:

*   a) Número de ciudades distintas en el fichero
*   b) Puntuación media por ciudad
*   c) Restaurante con mayor número de reviews
*   d) ¿Cuál es la puntuación media de los resturantes con precio $\$\$\$
*   e) ¿Cuál es la puntuación media así como el ranking medio de los resturantes con precio \"$$$$"?



In [ ]:
# a) Número de ciudades distintas en el fichero
data.select('City').distinct().count()

31

In [25]:
# b) Puntuación media por ciudad
data.groupby('City').agg({'Rating': 'mean'}).collect()

[Row(City='Madrid', avg(Rating)=3.796697684975034),
 Row(City='Prague', avg(Rating)=4.0134228187919465),
 Row(City='Edinburgh', avg(Rating)=4.056818181818182),
 Row(City='Lisbon', avg(Rating)=4.052128232758621),
 Row(City='Stockholm', avg(Rating)=3.8735282176207875),
 Row(City='Oslo', avg(Rating)=3.899384885764499),
 Row(City='Dublin', avg(Rating)=4.051150895140665),
 Row(City='Berlin', avg(Rating)=4.127020241644438),
 Row(City='London', avg(Rating)=3.9428962579860056),
 Row(City='Vienna', avg(Rating)=4.067983730389309),
 Row(City='Paris', avg(Rating)=3.948714285714286),
 Row(City='Hamburg', avg(Rating)=4.030507855316039),
 Row(City='Athens', avg(Rating)=4.207774049217003),
 Row(City='Lyon', avg(Rating)=3.9203821656050954),
 Row(City='Ljubljana', avg(Rating)=4.128205128205129),
 Row(City='Zurich', avg(Rating)=4.018495297805643),
 Row(City='Krakow', avg(Rating)=4.128812199036918),
 Row(City='Milan', avg(Rating)=3.808954748247291),
 Row(City='Barcelona', avg(Rating)=3.9668292056974206),


In [ ]:
# c) Restaurante con mayor número de reviews
max_value = data.agg({'NumberOfReviews' : 'max'}).collect()[0][0]

data.filter(data.NumberOfReviews == max_value).show()

+-----+------------+------+--------------------+-------+------+----------+---------------+
|Index|        Name|  City|        CuisineStyle|Ranking|Rating|PriceRange|NumberOfReviews|
+-----+------------+------+--------------------+-------+------+----------+---------------+
|   59|Hofbraeuhaus|Munich|['German', 'Bar',...|   60.0|   4.0|  $$ - $$$|        16478.0|
+-----+------------+------+--------------------+-------+------+----------+---------------+



In [ ]:
# d) ¿Cuál es la puntuación media de los resturantes con precio "$$$$"?
data.filter(data.PriceRange == "$$$$").agg({'Rating' : 'mean'}).show()

+-----------------+
|      avg(Rating)|
+-----------------+
|4.201684717208183|
+-----------------+



In [ ]:
# e) ¿Cuál es la puntuación media así como el ranking medio de los resturantes con precio "$$$$"?
data.groupby('PriceRange').agg({'Rating' : 'mean', 'Ranking' : 'mean'}).show()

+----------+------------------+------------------+
|PriceRange|      avg(Ranking)|       avg(Rating)|
+----------+------------------+------------------+
|      $$$$|1938.3900384985564| 4.201684717208183|
|      null| 4858.669976968226|3.9227876598797655|
|  $$ - $$$| 2923.015094836175| 3.984216958074826|
|         $| 3531.331459435626|4.0919419778280295|
+----------+------------------+------------------+



### Eliminar columnas del dataframe

In [29]:
data = data.drop('CuisineStyle')
data.show(3)

+-----+--------------------+---------+-------+------+----------+---------------+
|Index|                Name|     City|Ranking|Rating|PriceRange|NumberOfReviews|
+-----+--------------------+---------+-------+------+----------+---------------+
|    0|Martine of Martin...|Amsterdam|    1.0|   5.0|  $$ - $$$|          136.0|
|    1| De Silveren Spiegel|Amsterdam|    2.0|   4.5|      $$$$|          812.0|
|    2|             La Rive|Amsterdam|    3.0|   4.5|      $$$$|          567.0|
+-----+--------------------+---------+-------+------+----------+---------------+
only showing top 3 rows



### Missings y valores duplicados

Un valor missing es aquel que no está relleno. Estos valores suelen causar problemas en los análisis, por lo que es habitual su eliminación o imputación. Veamos algunos métodos destinados a ello.

In [ ]:
from pyspark.sql.functions import col,isnan, when, count
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]
   ).show()

+-----+----+----+-------+------+----------+---------------+
|Index|Name|City|Ranking|Rating|PriceRange|NumberOfReviews|
+-----+----+----+-------+------+----------+---------------+
|    0|   0|   0|   9651|  9630|     47855|          17344|
+-----+----+----+-------+------+----------+---------------+



In [30]:
data_no_nan = data.dropna(how = 'any', subset = 'Ranking')
data_no_nan.count()

115876

In [31]:
data.count()

125527

In [32]:
from pyspark.sql.functions import col,isnan, when, count
data_no_nan.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data_no_nan.columns]
   ).show()

+-----+----+----+-------+------+----------+---------------+
|Index|Name|City|Ranking|Rating|PriceRange|NumberOfReviews|
+-----+----+----+-------+------+----------+---------------+
|    0|   0|   0|      0|   125|     40379|           7721|
+-----+----+----+-------+------+----------+---------------+



Otra opción puede ser la de rellenar el valor de los missings, para así no perder filas.

In [33]:
ranking_mean = data.agg({'Rating' : 'mean'}).collect()[0][0]
ranking_mean

3.9874414350673444

In [ ]:
data = data.fillna(ranking_mean, subset = 'Rating')

In [ ]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]
   ).show()

+-----+----+----+-------+------+----------+---------------+
|Index|Name|City|Ranking|Rating|PriceRange|NumberOfReviews|
+-----+----+----+-------+------+----------+---------------+
|    0|   0|   0|   9651|     0|     47855|          17344|
+-----+----+----+-------+------+----------+---------------+



Respecto a las duplicidades en los registros, puede llegar a ser peligroso en cierto análisis. Para este fin tenemos el método *drop_duplicates*


In [35]:
ciudad_unica = data.drop_duplicates(subset=['City'])
ciudad_unica.show()

+-----+--------------------+----------+-------+------+----------+---------------+
|Index|                Name|      City|Ranking|Rating|PriceRange|NumberOfReviews|
+-----+--------------------+----------+-------+------+----------+---------------+
|    0|           Finnjavel|  Helsinki|    1.0|   4.5|      $$$$|          305.0|
|   21|            Albrecht|Bratislava|   22.0|   4.5|      $$$$|          175.0|
|    3|        Odprta Kuhna| Ljubljana|    4.0|   4.5|         $|          194.0|
|   80|       Oven Gran Via|    Madrid|   81.0|   4.5|      null|         1514.0|
|   91|La Casa de la Hav...|    Prague|   92.0|   5.0|      null|          276.0|
|    2|               ASPIC|     Paris|    3.0|   5.0|      $$$$|          427.0|
|    0|            La Fonte|    Zurich|    1.0|   4.5|  $$ - $$$|          704.0|
|    1|        Tonton Garby|  Brussels|    2.0|   5.0|         $|         1141.0|
|   23|Michael Neave's K...| Edinburgh|   24.0|   4.5|      null|          973.0|
|    1|         

### Escritura

Por último, podemos escribir el resultado de una consulta en un fichero CSV, JSON, etc.

In [37]:
rest_barc = data.filter(data.City == "Barcelona")
rest_barc.show(3)

+-----+------+---------+-------+------+----------+---------------+
|Index|  Name|     City|Ranking|Rating|PriceRange|NumberOfReviews|
+-----+------+---------+-------+------+----------+---------------+
|    0|   Uma|Barcelona|    1.0|   5.0|      $$$$|          792.0|
|    1| Viana|Barcelona|    2.0|   5.0|  $$ - $$$|         2707.0|
|    2|Blavis|Barcelona|    3.0|   5.0|  $$ - $$$|          643.0|
+-----+------+---------+-------+------+----------+---------------+
only showing top 3 rows



In [38]:
rest_barc.count()

8425

In [40]:
rest_barc.write.csv(data_path + 'rest_barc.csv', sep = ';')

In [41]:
rest_barc.write.json(data_path + 'rest_barc.json')